In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, balanced_accuracy_score
import numpy as np
import pickle



from utils.general_utils import get_outlier_info, get_outlier_val_counts

## TODO 
1. Stratified split
2. Handle class imbalance
3. Handle outliers

In [1]:
og_df = pd.read_csv('dataset/TrainDataset2023.csv')
og_df.rename(columns={'pCR (outcome)': 'pcr', 'RelapseFreeSurvival (outcome)': 'rfs'}, inplace=True)

NameError: name 'pd' is not defined

In [4]:
with open(f'dataset/dataset_v1/data.npy', 'rb') as f:
    imputed_arr = np.load(f)

In [5]:
imputed_arr.shape

(400, 55)

In [14]:
classification_y = imputed_arr[:,0]
regression_y = imputed_arr[:,1]
x_data =  imputed_arr[2:]

In [15]:
imputed_arr.shape, x_data.shape, regression_y.shape, classification_y.shape

((400, 55), (398, 55), (400,), (400,))

In [67]:
with open(f'dataset/dataset_v1/data.npy', 'wb') as f:
    np.save(f, imputed_arr)

In [16]:
train, test = train_test_split(imputed_arr, test_size=0.20)
train.shape, test.shape

((320, 55), (80, 55))

In [27]:
train_classification_y = train[:,0]
train_regression_y = train[:,1]

test_classification_y = test[:,0]
test_regression_y = test[:,1]

xTrain_data =  train[:,2:]
xTest_data =  test[:, 2:]

In [28]:
train_classification_y.shape, train_regression_y.shape, test_classification_y.shape, test_regression_y.shape, xTrain_data.shape, xTest_data.shape

((320,), (320,), (80,), (80,), (320, 53), (80, 53))

# Regression Task split and z-score normalization

In [29]:
scalerTrainX = StandardScaler()
scalerTrainY = StandardScaler()

trainX = scalerTrainX.fit_transform(xTrain_data)
trainY = scalerTrainY.fit_transform(train_regression_y.ravel())

testX = scalerTrainX.transform(xTest_data)
testY = scalerTrainY.transform(test_regression_y.ravel())

In [20]:
mean_absolute_error
balanced_accuracy_score

<function sklearn.metrics._classification.balanced_accuracy_score(y_true, y_pred, *, sample_weight=None, adjusted=False)>

# Model training - dry run
### Random Forest

In [54]:
from sklearn.ensemble import RandomForestRegressor
from utils.general_utils import save_model

In [44]:
regressor = RandomForestRegressor(random_state=0, verbose=1)


In [45]:
regressor.fit(trainX, trainY.ravel())

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.5s


RandomForestRegressor(random_state=0, verbose=1)

In [46]:
predicted = regressor.predict(testX)
test_mse_error = mean_squared_error(predicted, testY)
test_mae_error = mean_absolute_error(predicted, testY)
test_r2_error = r2_score(predicted, testY)

print(f'MSE error: {test_mse_error}')
print(f'MAE error: {test_mae_error}')
print(f'R2 error: {test_r2_error}')

MSE error: 0.9765851073017249
MAE error: 0.7654654184592893
R2 error: -5.57654164850365


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [59]:
def save_normalizer(path, Xnormalizer, Ynormalizer):
    
    print('Caution, saving normalizer in order!.. Check arguments')
    
    normalilzer_list = [Xnormalizer, Ynormalizer]
    name_list = ['scalerTrainX', 'scalerTrainY']
    
    for nlz, name in zip(normalilzer_list, name_list):
        with open(f'{path}/{name}.pkl','wb') as f:
            pickle.dump(nlz, f)
        
def load_normalizer(path):
    
    print('Caution, loading normalizer in order!.. Check arguments')
    
    name_list = ['scalerTrainX', 'scalerTrainY']
    normalilzer_list = []
    
    for name in name_list:
        with open(f'{path}/{name}.pkl','rb') as f:
            normalilzer_list.append(pickle.load(f))
            
    return normalilzer_list[0], normalilzer_list[1]
        
def save_dataset(xTrain, yTrain, xTest, yTest, path):
    print('Caution, saving dataset in order!.. Check arguments')
    data_list = [xTrain, yTrain, xTest, yTest]
    data_name = ['xTrain', 'yTrain', 'xTest', 'yTest']
    
    for arr, name in zip(data_list, data_name):
        with open(f'{path}/{name}.npy', 'wb') as f:
            np.save(f,arr)

def load_dataset(path):
    
    print('Caution, loading dataset in order!.. Check arguments')
    
    data_name = ['xTrain', 'yTrain', 'xTest', 'yTest']      
    data_list = []
    for name in data_name:
        with open(f'{path}/{name}.npy', 'rb') as f:
            data_list.append(np.load(f))
            
    return data_list[0], data_list[1], data_list[2], data_list[3]
            

In [61]:
save_normalizer('baseline_model/normalizer/', scalerTrainX, scalerTrainY)
save_dataset(xTrain=trainX, yTrain=trainY, xTest=testX, yTest=testY, path='baseline_model/dataset/')
save_model(model=regressor, save_path='baseline_model/model/', name="randomforest")

Caution, saving things in order!.. Check arguments


In [ ]:
# Make sure not changing the oridinal feature, binary feature 